In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

invasive-> 굳이 데이터프레임 ㄴㄴ 
이 대회는 주소 안에 정답값이 있는 형태

**교차 검증필요 -> 최대한 높게

In [ ]:
train=pd.read_csv('../input/state-farm-distracted-driver-detection/driver_imgs_list.csv')
train

In [ ]:
import glob
glob.glob('../input/state-farm-distracted-driver-detection/imgs/train/*/*')
train=pd.DataFrame({'path':glob.glob('../input/state-farm-distracted-driver-detection/imgs/train/*/*')})
pd.options.display.max_colwidth=999
train['target']=train['path'].apply(lambda x:x.split('/')[-2])
train

In [ ]:
train['target'].value_counts() # 클래스 별로 데이터 분포가 다른 상황, 이 비율에 맞추어서 평가셋을 만들어준다. 

In [ ]:
from sklearn.model_selection import StratifiedKFold #k-fold 쓰면 되는데 stratit 이건 정답클래스의 비율이 다를 때 사용한다.
skf=StratifiedKFold(n_splits=5,shuffle=True,random_state=7) #n_split 데이터 분할 수 
for train_index, valid_index in skf.split(train,train['target']): #train['target'] 컬럼의 카테고리를 기준으로, 비율에 맞게 
    x_train=train.iloc[train_index] #데이터에 접근해서 대괄호 진행하자, x_train이 20000개라 할 시 16000개가 들어가게된다. train에서 train_index만 들고 오겠다
    x_valid=train.iloc[valid_index]

교차 검증 시 5개로 나누어서 평가셋을 사용한다. 
또 새롭게 데이터를 뽑아서 나머지는 훈련용으로 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
idg = ImageDataGenerator(horizontal_flip=True)
train_generator = idg.flow_from_dataframe(x_train, x_col = "path", y_col = "target", target_size = (300,300), batch_size = 32)

In [ ]:
idg2=ImageDataGenerator()
vaild_generator=idg2.flow_from_dataframe(x_valid, x_col = "path", y_col = "target", target_size = (300,300), batch_size = 32)

In [ ]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [ ]:
from efficientnet.tfkeras import EfficientNetB1
from keras import Sequential
from keras.layers import * 
from keras.optimizers import SGD
model=Sequential()
model.add(EfficientNetB1(weights='imagenet',pooling='avg',include_top=False))
model.add(Dense(10,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer=SGD(lr=0.01,nesterov=True,momentum=0.9),metrics=['acc'])
from keras.callbacks import EarlyStopping,ModelCheckpoint, ReduceLROnPlateau
es=EarlyStopping(patience=5)
mc=ModelCheckpoint('best.h5',save_best_only=True)
RL=ReduceLROnPlateau(patience=3,factor=0.2) 
model.fit(train_generator,callbacks=[es,mc,RL],epochs=3)

In [ ]:
test=pd.DataFrame({'path':glob.glob('../input/state-farm-distracted-driver-detection/imgs/test/*')})
pd.options.display.max_colwidth=999
test

In [ ]:
idg3=ImageDataGenerator()
test_generator = idg2.flow_from_dataframe(test, x_col = "path", y_col = None, target_size = (300,300), batch_size = 32, class_mode=None, shuffle=False) 

In [ ]:
result=model.predict(test_generator)

In [ ]:
result

In [ ]:
sub=pd.read_csv('../input/state-farm-distracted-driver-detection/sample_submission.csv')
sub

In [ ]:
sub['img']=test['path'].apply(lambda x:x.split('/')[5])
sub

In [ ]:
sub.iloc[:, 1:] = result
sub

In [ ]:
sub.to_csv('state_farm3.csv',index=0)